Due to challenges faced trying to access the blockchain data en masse, i created a local instance of the lbry-explorer and this notebook leverages the mongoDB data source that exists as part of the lbry-explorer.

In [1]:
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import getpass
password = getpass.getpass()

········


In [20]:
def connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find()
    #print list(cursor)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df.drop("__v", axis=1)

Collections:
    addresses,
    markets,
    richlists,
    stats,
    txes

In [39]:
db = "explorerdb"

In [40]:
address_df = read_mongo(db, "addresses", host='localhost', port=27017, username="zargham", password=password)

In [41]:
address_df[address_df.a_id.map(lambda s: bool((s[0] != "b") & (s[0] !="r")))].head()

,a_id,balance,received,sent,txs
0,coinbase,0,0,6.310887e+14,"[{u'type': u'vin', u'addresses': u'5dda81f4219..."


In [42]:
labels = {"rEQKyb7nd7UUGyEEn5xRkk1fgXdTCf2ZCg": {"label":"LBRY Team", "type":"info", "url":"https://lbry.io/what#credits"},
      "bWbFyN8CJAghwavfRDuEfmv71swkapBcuy": {"label":"LBRY Hot Wallet", "type":"info", "url":""},
      "rVRMmGLZenVXpT1NwYMGcnwFc642kEgWTV": {"label":"Strategic Partners", "type":"info", "url":"https://lbry.io/what#credits"},
      "bLPbiXBp6Vr3NSnsHzDsLNzoy5o36re9Cz": {"label":"Investor", "type":"info", "url":"https://lbry.io/news/mysterious-100k-lbc-revealed"},
      "rTZF9RvfkLJ6hwCwERwFd7dppRMeddKjpe": {"label":"LBRY Team", "type":"info", "url":"https://lbry.io/what#credits"},
      "bVUrbCK8hcZ5XWti7b9eNxKEBxzc1rr393": {"label":"LBRY Hot Wallet", "type":"info", "url":""},
      "bHW58d37s1hBjj3wPBkn5zpCX3F8ZW3uWf": {"label":"LBRY Team Escrow (premine)", "type":"info", "url":"https://lbry.io/what#credits"},
      "r9DarmxyPjWkF7ocyxMzaNZN3a9gJvNTZJ": {"label":"Strategic Partners", "type":"info", "url":"https://lbry.io/what#credits"},
      "rMT5Sg8SyFP3ax2PRaweRCRZoMeYw4znEi": {"label":"Adoption Programs", "type":"info", "url":"https://lbry.io/what#credits"},
      "bDzuP6Sawe7tadVKR7dmPW1JFYK9ELnjra": {"label":"LBRY Hot Wallet", "type":"info", "url":""},
      "bFgohv4wprdCEtW2m5EVQyVNtxp4NxQMXi": {"label":"LBRY Hot Wallet", "type":"info", "url":""},
      "rRmURzvpHTysU4xUSp9CXeosBTbYfKs7n5": {"label":"Adoption Programs", "type":"info", "url":"https://lbry.io/what#credits"}
    }

In [46]:
def get_label(address):
    if address in labels.keys():
        return labels[address]["label"]
    else:
        return "unknown"

In [47]:
address_df["label"] = address_df.a_id.map(get_label)

In [48]:
address_df[address_df.label != "unknown"]

,a_id,balance,received,sent,txs,label
404,rEQKyb7nd7UUGyEEn5xRkk1fgXdTCf2ZCg,0.000000e+00,1.999000e+16,1.999000e+16,"[{u'type': u'vout', u'addresses': u'32f0ab6b2a...",LBRY Team
406,rMT5Sg8SyFP3ax2PRaweRCRZoMeYw4znEi,0.000000e+00,3.980000e+16,3.980000e+16,"[{u'type': u'vout', u'addresses': u'781c100a80...",Adoption Programs
410,r9DarmxyPjWkF7ocyxMzaNZN3a9gJvNTZJ,0.000000e+00,1.000000e+16,1.000000e+16,"[{u'type': u'vout', u'addresses': u'9600945091...",Strategic Partners
424,bHW58d37s1hBjj3wPBkn5zpCX3F8ZW3uWf,4.000000e+16,0.000000e+00,4.000000e+16,"[{u'type': u'vin', u'addresses': u'9a72043e11b...",LBRY Team Escrow (premine)
4057,bLPbiXBp6Vr3NSnsHzDsLNzoy5o36re9Cz,1.000090e+13,1.000090e+13,0.000000e+00,"[{u'type': u'vout', u'addresses': u'e2f9cb21d4...",Investor
4163,bFgohv4wprdCEtW2m5EVQyVNtxp4NxQMXi,0.000000e+00,7.352650e+13,7.352650e+13,"[{u'type': u'vout', u'addresses': u'7c77646740...",LBRY Hot Wallet
4833,bVUrbCK8hcZ5XWti7b9eNxKEBxzc1rr393,2.403880e+13,4.897770e+13,2.493890e+13,"[{u'type': u'vout', u'addresses': u'ed8fc03abd...",LBRY Hot Wallet
128905,bDzuP6Sawe7tadVKR7dmPW1JFYK9ELnjra,1.450000e+14,2.984199e+14,1.534199e+14,"[{u'type': u'vout', u'addresses': u'554d8fed52...",LBRY Hot Wallet
129116,bWbFyN8CJAghwavfRDuEfmv71swkapBcuy,0.000000e+00,8.419899e+12,8.419899e+12,"[{u'type': u'vout', u'addresses': u'b72ffc5bb9...",LBRY Hot Wallet
236796,rVRMmGLZenVXpT1NwYMGcnwFc642kEgWTV,1.000000e+16,1.000000e+16,1.000000e+06,"[{u'type': u'vout', u'addresses': u'73df395c9f...",Strategic Partners
